# RAG Chatbot using Ollama for LLM & Embeddings

In [1]:
import ollama
from langchain_community.llms import Ollama


In [3]:

llm = Ollama(
    model="mistral"
)

llm.invoke("Tell me a joke")

" Sure, here's a classic one for you:\n\nWhy was the math book sad?\nBecause it had too many problems.\n\nIf that didn't make you laugh, I apologize! Jokes can be quite subjective."

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = llm | parser
chain.invoke("Who owns microsoft?")

" Microsoft is a publicly-traded company, and no individual or entity owns the entire company. It is owned by millions of individual shareholders and institutional investors who own its common stock and other securities. The largest individual shareholder is currently Bill Gates, who co-founded the company. However, his ownership represents only a fraction of Microsoft's total shares outstanding."

In [5]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(
            context='''RAG is a general-purpose fine-tuning recipe for retrieval-augmented generation (RAG).''' , 
            question="What is full form of RAG?")

'Human: \nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: RAG is a general-purpose fine-tuning recipe for retrieval-augmented generation (RAG).\n\nQuestion: What is full form of RAG?\n'

In [10]:
chain = prompt | llm | parser
chain.invoke({
    "context": "RAG is a general-purpose fine-tuning recipe for retrieval-augmented generation (RAG).",
    "question": "What is full form of RAG?"
})

' I don\'t have access to or knowledge about the specific meanings given to acronyms outside of my training data. In this case, I can\'t confirm if RAG has a defined full form in this context as it appears to be an abbreviation for "Retrieval-augmented generation."'

In [11]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Who Moved My Cheese_.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 0}, page_content="Who Moved My Cheese? Summary\nBy  Spencer Johnson\nChange is the only certainty in life, so why do we avoid and even fear it?\nWe know that change is inevitable, yet we lack effective strategies to\ncome to terms with it. What if there was a roadmap to help us better\nnavigate change?\nWho Moved My Cheese  is a light-hearted fable that offers deep insights into\nadapting to change in multiple scenarios. In a simple story of the\ninevitability of change, two mice and two little people navigate living in a\nmaze and maintaining their cheese supplies. The characters reveal that\nwe can either accept change, or complain about it.\nThis summary takes a brief look at Dr. Johnson's story. It tackles the\ncomplexity of how we react to change, as a means to inspire us to be\ncourageous in the face of it. Growth often emerges because of adversity,\nand sometimes unexpected scenarios arise when we least expect the

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(pages)[:5]


[Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 0}, page_content='Who Moved My Cheese? Summary\nBy  Spencer Johnson'),
 Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 0}, page_content='By  Spencer Johnson\nChange is the only certainty in life, so why do we avoid and even fear it?'),
 Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 0}, page_content='We know that change is inevitable, yet we lack effective strategies to'),
 Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 0}, page_content='come to terms with it. What if there was a roadmap to help us better\nnavigate change?'),
 Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 0}, page_content='navigate change?\nWho Moved My Cheese  is a light-hearted fable that offers deep insights into')]

In [13]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(pages)

In [14]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = (
    OllamaEmbeddings(model='mistral')
)  

text = "This is a test document."

In [15]:
query_result = embeddings.embed_query(text)
query_result[:5]

[1.8355729579925537,
 -2.2049717903137207,
 0.10405337810516357,
 -10.774480819702148,
 1.3713029623031616]

In [16]:
print(f"Embedding length: {len(query_result)}")

Embedding length: 4096


In [19]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(documents=documents, embedding=embeddings)

In [ ]:
db.save_local("faiss_index")

In [20]:
retriver = db.as_retriever()

In [ ]:
db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [22]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriver, question=RunnablePassthrough())
setup.invoke("Who is author of book?")

{'context': [Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 3}, page_content='monitoring the initial cheese scenario. They remained action-focused,\nrather than becoming caught in over-analysis and stale resentments.'),
  Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 6}, page_content="keep things simple. The two little people's complex brains and human\nemotions, tend to complicate things. Whatever parts of us we choose to\nembrace, or choose to ignore, we all have something in common. All of us\nhave the universal need to find our way around the maze, and succeed in\nchanging times.\nIn Conclusion"),
  Document(metadata={'source': 'Who Moved My Cheese_.pdf', 'page': 1}, page_content="might see them. Eventually, Haw discovers a new station with an\nabundance of cheese, and he reunites with Sniff and Scurry. They have\nbeen there for some time, enjoying the fruits of their labor.\nOne of the book's central messages is that Haw's notes on the wall ser

In [23]:
chain = (
    {"context": db.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

In [24]:
chain.invoke("What is names of mice")

' The names of the mice are Sniff and Scurry.'

In [25]:
chain.invoke("can you tell me learning of this book?")

' The learning from the book "Who Moved My Cheese" revolves around adapting to change and navigating through the challenges it brings. It emphasizes the importance of being action-focused, rather than getting stuck in over-analysis or stale resentments. The book encourages keeping things simple and embracing change as the only constant, suggesting that fear is the main obstacle preventing us from moving into the unknown. The author suggests asking oneself what one would do if fear weren\'t holding them back. The central message is to find ways around the maze of life, succeed in changing times, and learn how to anticipate, adapt, and ultimately enjoy the journey of transformation.'

In [26]:
chain.invoke("what is mean by keep things simple?")

' In the context provided, "keep things simple" means minimizing complexity and simplifying situations or problems. This is because complex brains and human emotions tend to complicate things, as indicated in the text from page 6 of "Who Moved My Cheese". The author suggests that despite our complexities, we all have something in common - the need to navigate changes successfully. Thus, maintaining a simple approach can help us move forward more easily in changing times.'